# Harnessing Matplotlib & Astropy

## Matplotlib.pyplot

### General tips:
- **Don't overengineer your plotting code.** This means you don't need to add a million keywords anticipating every possible tweak your users might wish to make. Focus on making it easy for your users to modify the code themselves. Make sure you return `Figure` and `Axes` objects (more on this below) to your user so they can easily modify plots.
- **Apply principles of good design.** This means using color palettes with lots of contrast, making your fonts large and easy-to-read, and making sure your plots are colorblind friendly. (There are several great colorblindness simulators out there, for example [this one](https://www.color-blindness.com/coblis-color-blindness-simulator/).) It's also good to differentiate plot elements in more than one way (ex: one line is black and dashed, another is red and solid). 

### This Tutorial:

`matplotlib.pyplot` is a powerful graphics module often used by Python developers. (Check out [this gallery](https://matplotlib.org/gallery.html) to get a sense of what you can do with pyplot.) This is by no means a comprehensive tutorial, but it will point out a few important things about `matplotlib.pyplot` objects and give advice about incorporating plotting functionality in your open-source astronomy software.

There are other (excellent) plotting tools in common practice in astronomy, especially plotly and bokeh. We encourage you to look into these tools as well, but we'll only discuss `matplotlib.pyplot` here.

### Tutorial Starts Here.

`matplotlib.pyplot` relies on the concept of *changing state*. An excellent example of this concept is the `matplotlib.Rcparams` object, which is like a dictionary of attributes that control the "default" behaviors of matplotlib plots. Let's use the `update()` method of this object to change the default font size:


In [ ]:
from matplotlib import rcParams
import matplotlib.pyplot as plt

# make a plot
plt.figure()
plt.xlabel('foo'); plt.ylabel('bar')

# update the current state
rcParams.update({'font.size': 22})
print('The current font size is {}.'.format(rcParams['font.size']))

# make another plot
plt.figure()
plt.xlabel('foo'); plt.ylabel('bar', fontsize=10)

# update the current state again
rcParams.update({'font.size': 12})
print('Now, the current font size is {}.'.format(rcParams['font.size']))

As you can see, `matplotlib` decides on a default font size by checking the *current* value of the state parameter `rcParams['font.size']`, but this is always overwritten if you pass in a keyword (in this case, the font size of the y axis label).

The basic "unit" of `matplotlib.pyplot` is [the `Figure` object](https://matplotlib.org/3.2.1/api/_as_genmatplotlib.figure.Figure.html#matplotlib.figure.Figure), which acts as container for all the other plot elements. Let's look at a few different ways to create a `Figure` object:

In [ ]:
import numpy as np

# create some sample data
x = np.array([1, 2, 3, 4])
y = np.array([1, 4, 9, 16])
y_error = np.array([5, 1.2, 2, 5])

fig = plt.figure() # initializes a `Figure` object
plt.errorbar(      # plots onto the current `Figure`
    x, y, y_error, ls='', fmt="*"  
)  

print(type(fig))

The current figure changes (i.e. the *state* is updated) when you initialize a new `Figure` object. Unless you specify a particular `Figure` object, `plt.___()` functions will always add to the current `Figure` object. That is a nice feature-- it makes it easy to add multiple types of plots onto the same figure.

Let's look at another way to initialize a new `Figure` object:

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

ax[0].plot(x, y)
ax[1].plot(x, y)
ax[1].set_yscale('log')

print(type(fig))
print(type(ax))
print(type(ax[0]))

Here we've encounter another kind of object: [the `Axes` object](https://matplotlib.org/api/axes_api.html#matplotlib.axes.Axes). These objects are containers for all the plot elements *in a particular figure panel*. `Axes` are attributes of `Figure` objects:

In [ ]:
fig.axes[0].set_xlim(1, 3)
fig

Two useful methods to keep in mind are `plt.gca()` and `plt.sca()` (get/set current `Axes`). Let's see how they work:

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2)

mystery_axes = plt.gca()              # get current Axes
mystery_axes.scatter(x, y, color='k') # acts on `mystery_axes`

plt.sca(ax[0,0])        # set current Axes
plt.plot(x, y, ls='-.') # acts on current Axes
plt.tight_layout()      # prettify

#### Activity #1 

Let's practice working with `Figure` and `Axes` objects. Run the code snippet below to generate some orbits and plot them using `orbitize!`

In [ ]:
import orbitize.driver

# set up fit
myDriver = orbitize.driver.Driver(
    '{}/GJ504.csv'.format(orbitize.DATADIR),
    'OFTI', 1, 1.22, 56.95, mass_err=0.08, plx_err=0.26
)
s = myDriver.sampler

# run orbitize!
orbits = s.run_sampler(100)
myResults = s.results

# make plot
epochs = myDriver.system.data_table['epoch']
orbit_figure = myResults.plot_orbits(
    start_mjd=epochs[0]
)

Notice that the last line of this snippet returns the `Figure` object used to make the plot above. This is good practice-- it lets your users more easily customize figures, just as you're about to do!

Tasks:
- Change the x-axis limits of the top right panel to 2011-2015.
- Add a black star at (0, 0) in the leftmost panel.
- Change the dpi (dots per inch, a measure of resolution) of the Figure to 250.
- Save the figure to your Desktop using [`plt.savefig()`](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.savefig.html). 

### Activity #2

Use gridspec (Google it!) to create a basic [corner plot](https://corner.readthedocs.io/en/latest/pages/quickstart.html). Plot the samples generated in the code snippet below. Your corner plot must:
- display histograms along the diagonal
- display a contour plot along the non-diagonal
- handle 2-dimensional arrays

Your corner plot may, if you have time/interest:
- be super pretty
- handle N-dimensional arrays

In [ ]:
import numpy as np

ndim, nsamples = 2, 10000
samples = np.random.randn(ndim * nsamples)

print(samples)

## (Optional) Activity #3

Read about [interactive plots](https://matplotlib.org/3.2.1/users/interactive.html) in matplotlib and/or [making animations](https://matplotlib.org/3.2.1/api/animation_api.html) in matplotlib. Make some cool figures. :)


# Astropy

`astropy` is full of useful stuff for astronomers (perhaps unsurprisingly). The main point I want to drive home with this tutorial is **DON'T REINVENT THE WHEEL**. Before you write a new function, especially one that performs a common astronomical task, check whether an `astropy` developer wrote it for you.

Again, this tutorial is far from exhaustive. Check out a more complete list of `astropy` tutorials [here](http://learn.astropy.org/tutorials.html).

Below are some examples of a few particularly useful `astropy` features. 

One of my favorite things about `astropy` is its ability to handle units and fundamental constants. Let's calculate the Earth's orbital velocity:

In [ ]:
import numpy as np
from astropy import constants as cst, units as u 

v_Earth = np.sqrt(
    cst.G * 
    cst.M_sun / 
    (1 * u.au)
)

print(v_Earth)

Well those units aren't very helpful. Let's convert them to more physical units:

In [ ]:
v_Earth_cgs = v_Earth.cgs
v_Earth_km_s = v_Earth.to(u.km/u.s)

print("Earth's orbital velocity: {:.1f}.".format(v_Earth_cgs))
print("Another way to put it: {:.1f}.".format(v_Earth_km_s))

Next, let's do an example with the bane of every astronomer's existence: time.

In [ ]:
from astropy.time import Time 

my_birthday = Time('1994-11-04', format='iso')

print('As a fractional year: {}.'.format(my_birthday.decimalyear))
print('In JD: {}.'.format(my_birthday.jd))
print('In MJD: {}.'.format(my_birthday.mjd))

Next, a quick example with the second-worst bane of every astronomer's existence: coordinate systems. `astropy` has a great `coordinates` module that enables easy translation between coordinate systems:

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u

# input coordinates as hour angle and degree
my_coordinate = SkyCoord('00:42.5 +41:12', unit=(u.hourangle, u.deg))

# access right ascension
print(my_coordinate)

# convert to galactocentric coordinates
print(my_coordinate.galactic)

As a final example, we will use astropy to read a .fits file and access its contents. When using `fits.open()`, it's important to close the file after you're finished with it to avoid taking up memory.

In [ ]:
from astropy.io import fits

fits_image_filename = fits.util.get_testdata_filepath('test0.fits')
hdul = fits.open(fits_image_filename)

# print a summary of the file contents
hdul.info()

# print the contents of the .fits header
hdul[0].header

# close the file
hdul.close()

You can avoid having to close the file yourself by using the `with` command:

In [ ]:
with fits.open(fits_image_filename) as hdul:
    hdul.info()     # file is automatically closed when loop exits

### Activity #1

Google around and learn about how the `astropy.units` module handles parallax. Use `astropy` to convert from a proper motion of 30 arcseconds/yr to physical units (e.g. au/yr). Assume the distance to the object is 10 pc.


### Activity #2

The code below has an error. Good error messages will tell you exactly what went wrong and where it happened, but it's very difficult to write error messages that will be useful in every situation. When faced with an error message you don't understand, it's good practice to Google it (literally just copy the error message and paste it into Google) to see if anyone else has encountered that error. StackOverflow is especially helpful. Try it with the following error message, and fix the code so that it gives you the electrostatic force in Dynes.


In [ ]:
from astropy import units as u, constants as cst

# calculate electrostatic force between proton and electron
force = cst.e**2 / (1 * u.angstrom)**2

### Activity #3

Let's play with the .fits file we accessed above a little more. 
- Access the readout time from `my_file` below and store it in a variable `readtime`. 
- Figure out how to access the data. Make a plot of one of the science exposures (use `pyplot.imshow()`).

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt

fits_image_filename = fits.util.get_testdata_filepath('test0.fits')
hdul = fits.open(fits_image_filename)

hdul.info()
my_file = hdul[0].header

### Your code here ###


